In [1]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "cv_generator"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"

In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import json
from pathlib import Path
import datetime

time_start = datetime.datetime.now()

# Définir le modèle Pydantic pour la structure attendue
type Experiences = list[dict[str, str]]
type Education = list[dict[str, str]]

class ProfileData(BaseModel):
    experiences: Experiences = Field(description="Liste des expériences professionnelles du candidat")
    education: Education = Field(description="Liste des formations académiques du candidat")

# Initialisation du modèle LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Définir le parser JSON
parser = JsonOutputParser(pydantic_object=ProfileData)

In [3]:
import json
from pathlib import Path
# Charger le contenu du fichier source
source_file = Path("source_brut.txt")
if not source_file.exists():
    raise FileNotFoundError("Le fichier source_brut.txt est introuvable.")

with source_file.open("r", encoding="utf-8") as f:
    source_text = f.read()

In [4]:
from langchain_community.callbacks import get_openai_callback

# Création du prompt en intégrant les instructions de formatage
prompt = PromptTemplate(
    template="""
    Analyse le texte suivant décrivant un profil candidat et génère un JSON structuré.
    Le JSON doit contenir deux clés : "experiences" et "education", avec les champs suivants :
    
    - Pour chaque expérience :
      - "intitule": Intitulé du poste
      - "dates": Période d'emploi
      - "etablissement": Nom de l'entreprise
      - "lieu": Localisation
      - "description": L'intégralité des informations disponibles concernant cette expérience, sans résumé ni reformulation.
    
    - Pour chaque formation :
      - "intitule": Nom du diplôme
      - "dates": Période de formation
      - "etablissement": Nom de l'institution
      - "lieu": Localisation
      - "description": L'intégralité des informations disponibles sur cette formation, sans résumé ni reformulation.
    
    {format_instructions}
    
    Texte source :
    """
    "{source}"
    """
    """,
    input_variables=["source"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Exécuter le prompt avec le modèle
json_chain = prompt | llm | parser


# Exécution de la chaîne en capturant le coût via le callback
with get_openai_callback() as cb:
    json_output = json_chain.invoke({"source": source_text})



In [5]:
# Ajout des informations de coût dans le JSON
json_output["total_tokens"] = cb.total_tokens
json_output["prompt_tokens"] = cb.prompt_tokens
json_output["completion_tokens"] = cb.completion_tokens
json_output["total_cost"] = cb.total_cost

# Sauvegarde du résultat dans un fichier JSON

# Calcul du temps d'exécution
time_diff = datetime.datetime.now() - time_start
execution_time_sec = time_diff.total_seconds()


#on ajoute des informations supplémentaires sur le run
#on ajoute le nom du fichier texte source, la date et l'heure, "direct" (le nom de cette méthode) et le nom du modèle utilisé
json_output["source_file"] = source_file.name
json_output["date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
json_output["method"] = "direct"
json_output["model"] = "gpt-4o-mini"
json_output["execution_time"] = execution_time_sec

output_file = Path(f"output_dir_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.json")
with output_file.open("w", encoding="utf-8") as f:
    json.dump(json_output, f, indent=2, ensure_ascii=False)

print("JSON structuré et enrichi généré avec succès et sauvegardé dans output.json")


JSON structuré et enrichi généré avec succès et sauvegardé dans output.json
